# EnsembleAgeStaticTop

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [16]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [17]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.EnsembleAgeStaticTop)

class EnsembleAgeStaticTop(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        return x



In [18]:
model = pya.models.EnsembleAgeStaticTop()

## Define clock metadata

In [19]:
model.metadata["clock_name"] = 'ensembleagestatictop'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Mus musculus'
model.metadata["year"] = 2025
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Haghani, A., Lu, A.T., Yan, Q. et al. EnsembleAge: enhancing epigenetic age assessment with a multi-clock framework. GeroScience (2025)"
model.metadata["doi"] = 'https://doi.org/10.1007/s11357-025-01808-1'
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download directly with curl

In [20]:
supplementary_url = "https://static-content.springer.com/esm/art%3A10.1007%2Fs11357-025-01808-1/MediaObjects/11357_2025_1808_MOESM1_ESM.xlsx"
supplementary_file_name = "coefficients.xlsx"
os.system(f"curl -o {supplementary_file_name} {supplementary_url}")

0

## Load features

#### From Excel file

In [21]:
df = pd.read_excel('coefficients.xlsx', sheet_name='Table S3. Ensemble.Static')
df = df[df['EnsembleAge.Static.Top_ElasticNoAgeTraf'] != 0]
model.features = df['CGid'][1:].tolist()

## Load weights into base model

In [22]:
weights = torch.tensor(df['EnsembleAge.Static.Top_ElasticNoAgeTraf'][1:].tolist()).unsqueeze(0).float()
intercept = torch.tensor([df['EnsembleAge.Static.Top_ElasticNoAgeTraf'][0]]).float()

#### Linear model

In [23]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [24]:
model.reference_values = None

## Load preprocess and postprocess objects

In [25]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [26]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [27]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Haghani, A., Lu, A.T., Yan, Q. et al. EnsembleAge: enhancing '
             'epigenetic age assessment with a multi-clock framework. '
             'GeroScience (2025)',
 'clock_name': 'ensembleagestatictop',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1007/s11357-025-01808-1',
 'notes': None,
 'research_only': None,
 'species': 'Mus musculus',
 'version': None,
 'year': 2025}
reference_values: None
preprocess_name: None
preprocess_dependencies: None
postprocess_name: None
postprocess_dependencies: None
features: ['cg00045149', 'cg00091964', 'cg00117955', 'cg00158333', 'cg00181698', 'cg00243488', 'cg00258262', 'cg00259292', 'cg00407148', 'cg00409684', 'cg00493887', 'cg00553880', 'cg00578937', 'cg00659065', 'cg00896209', 'cg00929113', 'cg00946032', 'cg00957561', 'cg01024113', 'cg01039540', 'cg0106157

## Basic test

In [28]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ -9.4919],
        [ -4.8999],
        [-11.6078],
        [  5.9647],
        [ 23.9748],
        [  3.0499],
        [-12.5388],
        [  4.6096],
        [  5.1646],
        [-10.7823]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [29]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [30]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: coefficients.xlsx
